# Upload data from Excel

### Import enviPath packages, getpass for passwords, and pandas for data cleaning

In [43]:
from enviPath_python.enviPath import *
from enviPath_python.objects import *
import getpass
import pandas as pd

### Access password-protected package

In [44]:
if __name__ == "__main__":
    eP = enviPath('https://envipath.org')
    username = "slr257"
    eP.login(username, getpass.getpass(prompt="Password: ", stream=None))

    pkg = Package(eP.requester, id="https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e") # Currently the playground package

Password:  ········


### Read in excel data from template using pandas
each sheet is a separate dataframe

In [45]:
folder_name = "PFAS_excel_uploadfiles/2024_FangB/"
file_name = "6-2FTNO_v2.xlsx"

In [50]:
# read in excel data from template using pandas
cmpds = pd.read_excel(folder_name + file_name, sheet_name = "Compounds")
conn = pd.read_excel(folder_name + file_name, sheet_name = "Connectivity")
scn = pd.read_excel(folder_name + file_name, sheet_name = "Scenario")
kin_conf = pd.read_excel(folder_name + file_name, sheet_name = "Kinetics_Confidence")

Scenario_Name = "Fang, B. et al, 2024"
DOI = "10.1021/acs.est.3c05506"

# running this generates a warning because there are drop-down lists in excel from the "data validation" extension that do not carry over into the dataframes here

C:\Users\strich\AppData\Local\anaconda3\envs\enviPath_test_12324\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\strich\AppData\Local\anaconda3\envs\enviPath_test_12324\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\strich\AppData\Local\anaconda3\envs\enviPath_test_12324\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### Add Compounds

First work with the compounds data frame to upload nodes

In [51]:
# define parent node
parent = cmpds.loc[cmpds['Type'] == 'Parent']

# creat root node
pw = Pathway.create(pkg, smiles = parent.SMILES[0], name = parent.Name[0], root_node_only = True)

# library with SMILES and URLs for each node
map_smiles_to_URL = {"SMILES": [parent.SMILES[0]], "URL": [pw.get_nodes()[0].get_id()]} 

# related nodes
TPs = cmpds.loc[cmpds['Type'] != 'Parent']

for index in TPs.index:
    #if Type!="Parent":
        #Create Node
    row = TPs.loc[index]
    n = Node.create(pw, smiles = row.SMILES, name = row.Name)
    pw.add_node(smiles = row.SMILES, name = row.Name)
    # print(n.get_smiles() + " is not the same as " + row.SMILES )
    map_smiles_to_URL["SMILES"].append(row.SMILES)
    map_smiles_to_URL["URL"].append(n.get_id())
    print(f"Accessing compound {row.Name} with SMILES {row.SMILES}")
    
temporary_df = pd.DataFrame.from_dict(map_smiles_to_URL, orient="columns")
cmpds = cmpds.merge(temporary_df, left_on="SMILES", right_on="SMILES")
cmpds.head()

Accessing compound 6:2 FTAA with SMILES FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN(C)C)(F)F)(F)F)(F)F)(F)F)(F)F)F
Accessing compound 6:2 FTUAA with SMILES FC(F)(C(C(C(C(C(CCS(=O)(=O)NCC/C=[N+](\C)/C)(F)F)(F)F)(F)F)(F)F)(F)F)F
Accessing compound 6:2 FTSeAm with SMILES FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCNC)(F)F)(F)F)(F)F)(F)F)(F)F)F
Accessing compound 6:2 FTPrAm with SMILES FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN)(F)F)(F)F)(F)F)(F)F)(F)F)F
Accessing compound 6:2 FTSAm-PrOH with SMILES FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(NCCCO)(=O)=O)F
Accessing compound 6:2 FTSAm-PrA with SMILES FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(NCCC(O)=O)(=O)=O)F
Accessing compound N-Me-6:2 FTSAm-PrA with SMILES FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N(C)CCC(O)=O)(=O)=O)F
Accessing compound N-Et-6:2 FTSAm with SMILES FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(NCC)(=O)=O)F
Accessing compound 6:2 FTSAm-EtOH with SMILES FC(C(F)(C(F)(C(F)(C(F)(C(F)(CCS(NCCO)(=O)=O)F)F)F)F)F)(F)F
Accessing compound 6:2 FTSAm-AcAL with

,Type,Name,SMILES,URL
0,Parent,6:2 FTNO,C[N+](C)(CCCNS(=O)(=O)CCC(C(C(C(C(C(F)(F)F)(F)...,https://envipath.org/package/822cc087-5b9b-45d...
1,TP,6:2 FTAA,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN(C)C)(F)F)(F)F...,https://envipath.org/package/822cc087-5b9b-45d...
2,TP,6:2 FTUAA,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCC/C=[N+](\C)/C)(F...,https://envipath.org/package/822cc087-5b9b-45d...
3,TP,6:2 FTSeAm,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCNC)(F)F)(F)F)(F...,https://envipath.org/package/822cc087-5b9b-45d...
4,TP,6:2 FTPrAm,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN)(F)F)(F)F)(F)...,https://envipath.org/package/822cc087-5b9b-45d...


In [52]:
kin_conf = kin_conf.merge(temporary_df, left_on = "SMILES", right_on = "SMILES")
kin_conf

,Compound,SMILES,Model,Parameter,Value,Value Unit,R2,Confidence Level_PFAS,Confidence Level_Schy,URL
0,6:2 FTNO,C[N+](C)(CCCNS(=O)(=O)CCC(C(C(C(C(C(F)(F)F)(F)...,SFO,Half-life,1.2,d,0.9976,1a,1,https://envipath.org/package/822cc087-5b9b-45d...
1,6:2 FTAA,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN(C)C)(F)F)(F)F...,SFO,Half-life,11.5,d,0.9041,2a,2,https://envipath.org/package/822cc087-5b9b-45d...
2,6:2 FTUAA,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCC/C=[N+](\C)/C)(F...,NaN,NaN,NaN,NaN,NaN,2b,2,https://envipath.org/package/822cc087-5b9b-45d...
3,6:2 FTSeAm,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCNC)(F)F)(F)F)(F...,NaN,NaN,NaN,NaN,NaN,2a,2,https://envipath.org/package/822cc087-5b9b-45d...
4,6:2 FTPrAm,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN)(F)F)(F)F)(F)...,NaN,NaN,NaN,NaN,NaN,2c,2,https://envipath.org/package/822cc087-5b9b-45d...
5,6:2 FTSAm-PrOH,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,NaN,NaN,NaN,NaN,NaN,2a,2,https://envipath.org/package/822cc087-5b9b-45d...
6,6:2 FTSAm-PrA,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,NaN,NaN,NaN,NaN,NaN,2a,2,https://envipath.org/package/822cc087-5b9b-45d...
7,N-Me-6:2 FTSAm-PrA,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,NaN,NaN,NaN,NaN,NaN,3a,3,https://envipath.org/package/822cc087-5b9b-45d...
8,N-Et-6:2 FTSAm,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,NaN,NaN,NaN,NaN,NaN,2b,2,https://envipath.org/package/822cc087-5b9b-45d...
9,6:2 FTSAm-EtOH,FC(C(F)(C(F)(C(F)(C(F)(C(F)(CCS(NCCO)(=O)=O)F)...,NaN,NaN,NaN,NaN,NaN,2b,2,https://envipath.org/package/822cc087-5b9b-45d...


In [53]:
conn = pd.read_excel(folder_name + file_name, sheet_name = "Connectivity")
conn

C:\Users\strich\AppData\Local\anaconda3\envs\enviPath_test_12324\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Reactant,Reactant_SMILES,Product,Product_SMILES,Multistep
0,6:2 FTNO,C[N+](C)(CCCNS(=O)(=O)CCC(C(C(C(C(C(F)(F)F)(F)...,6:2 FTAA,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN(C)C)(F)F)(F)F...,NaN
1,6:2 FTNO,C[N+](C)(CCCNS(=O)(=O)CCC(C(C(C(C(C(F)(F)F)(F)...,6:2 FTUAA,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCC/C=[N+](\C)/C)(F...,NaN
2,6:2 FTNO,C[N+](C)(CCCNS(=O)(=O)CCC(C(C(C(C(C(F)(F)F)(F)...,6:2 FTSAm-PrOH,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,NaN
3,6:2 FTUAA,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCC/C=[N+](\C)/C)(F...,6:2 FTSeAm,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCNC)(F)F)(F)F)(F...,NaN
4,6:2 FTAA,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN(C)C)(F)F)(F)F...,6:2 FTSeAm,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCNC)(F)F)(F)F)(F...,NaN
5,6:2 FTSeAm,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCNC)(F)F)(F)F)(F...,6:2 FTPrAm,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN)(F)F)(F)F)(F)...,NaN
6,6:2 FTPrAm,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN)(F)F)(F)F)(F)...,6:2 FTSAm-PrOH,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,NaN
7,6:2 FTSAm-PrOH,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,6:2 FTSAm-PrA,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,multistep
8,6:2 FTSAm-PrA,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,N-Me-6:2 FTSAm-PrA,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,NaN
9,6:2 FTSAm-PrA,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,6:2 FTSAm,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,NaN


In [54]:
# now to add connectivity using edge info
for index in conn.index:
    #if Type!="Parent":
        #Create Node
    row = conn.loc[index]
    #pw.add_node(smiles = row.SMILES, name = row.Name)
    #print(f"Reactant smiles {row.Reactant_SMILES} with product smiles {row.Product_SMILES}")
    edge = row.Reactant_SMILES + '>>' + row.Product_SMILES

    # boolean value to add multistep attribute
    if pd.isnull(row.Multistep) == False:
        multi = row.Multistep
    else:
        multi = False
    print(edge)
    pw.add_edge(smirks = edge, multistep = multi)
# access connectivity dataframe

# concatenate >> between reactants and products

C[N+](C)(CCCNS(=O)(=O)CCC(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F)[O-]>>FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN(C)C)(F)F)(F)F)(F)F)(F)F)(F)F)F
C[N+](C)(CCCNS(=O)(=O)CCC(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F)[O-]>>FC(F)(C(C(C(C(C(CCS(=O)(=O)NCC/C=[N+](\C)/C)(F)F)(F)F)(F)F)(F)F)(F)F)F
C[N+](C)(CCCNS(=O)(=O)CCC(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F)[O-]>>FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(NCCCO)(=O)=O)F
FC(F)(C(C(C(C(C(CCS(=O)(=O)NCC/C=[N+](\C)/C)(F)F)(F)F)(F)F)(F)F)(F)F)F>>FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCNC)(F)F)(F)F)(F)F)(F)F)(F)F)F
FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN(C)C)(F)F)(F)F)(F)F)(F)F)(F)F)F>>FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCNC)(F)F)(F)F)(F)F)(F)F)(F)F)F
FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCNC)(F)F)(F)F)(F)F)(F)F)(F)F)F>>FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN)(F)F)(F)F)(F)F)(F)F)(F)F)F
FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN)(F)F)(F)F)(F)F)(F)F)(F)F)F>>FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(NCCCO)(=O)=O)F
FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(NCCCO)(=O)=O)F>>FC(F)(C(F)(F)C(F)(F)

### Create Main Scenario Object

In [55]:
# main scenario - aka "base"
base = scn[(scn['Environment Type'] == 'Activated Sludge')]
base


,Environment Type,Parameter,Value,Value Unit
0,Activated Sludge,pH,7.8 -8.5,NaN
1,Activated Sludge,Temperature,25,C
2,Activated Sludge,Name of Sampling Location,"Tianjin, China",NaN
3,Activated Sludge,Redox condition,aerob,NaN
4,Activated Sludge,Addition of nutrients,"sodium acetate, ammonium chloride",NaN
5,Activated Sludge,Biological treatment technology,other,NaN
6,Activated Sludge,Bioreactor,100 mL glass serum bottles,NaN
7,Activated Sludge,Inoculum source,aerobic sludge from aerobic tank,NaN
8,Activated Sludge,Purpose of WWTP,municipal WW,NaN
9,Activated Sludge,Solvent for compound addition,MeOH,NaN


### Checking Scenario Information and mating to enviPath objects

In [56]:
# this only works with the other fils is named utils...
import utils

# need to reload package when it's updated
from importlib import reload
reload(utils)

<module 'utils' from 'C:\\Users\\strich\\OneDrive - Universität Zürich UZH\\Documents\\data curation - EAWAG-PFAS package\\utils.py'>

### subset dictionary of functions to choose which functions to run to add AI objects

In [57]:
subset_dict = {k: utils.dict_ai[k] for k in base["Parameter"]}
print(subset_dict)

{'pH': <class 'enviPath_python.objects.AcidityAdditionalInformation'>, 'Temperature': <class 'enviPath_python.objects.TemperatureAdditionalInformation'>, 'Name of Sampling Location': <class 'enviPath_python.objects.LocationAdditionalInformation'>, 'Redox condition': <class 'enviPath_python.objects.RedoxAdditionalInformation'>, 'Addition of nutrients': <class 'enviPath_python.objects.NutrientsAdditionalInformation'>, 'Biological treatment technology': <class 'enviPath_python.objects.BiologicalTreatmentTechnologyAdditionalInformation'>, 'Bioreactor': <class 'enviPath_python.objects.BioreactorAdditionalInformation'>, 'Inoculum source': <class 'enviPath_python.objects.InoculumSourceAdditionalInformation'>, 'Purpose of WWTP': <class 'enviPath_python.objects.PurposeOfWWTPAdditionalInformation'>, 'Solvent for compound addition': <class 'enviPath_python.objects.SolventForCompoundSolutionAdditionalInformation'>, 'Type of Aeration': <class 'enviPath_python.objects.TypeOfAerationAdditionalInforma

In [58]:
# This function is really nice!!
# this should only run the add function if the variable k is in the function dictionary
additional_info_list = []
for k in subset_dict.keys():
    # print(k)
    if k in utils.dict_fun:
        utils.dict_fun[k](base, additional_info_list)
        # print(additional_info_list)
    else:
        pass
        

### Create a scenario object in enviPath

In [59]:
main_scenario = Scenario.create(pkg, name = Scenario_Name, description = "Reference:" + DOI ,additional_information=additional_info_list)

In [60]:
# add scenario to pathway
pw.add_scenario(main_scenario)

In [40]:
# this does not work yet because the root node is not always the first one listed in this pathway
# # add scenario to root node
# pw.get_nodes()[0].add_scenario(main_scenario)

In [61]:
pw.get_nodes()

[Node: 6:2 FTPrAm (https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/6df699f1-e23f-4245-bd10-286528c45feb/node/05de2d8f-8465-44c8-a033-b07333a21b81),
 Node: 6:2 FTSAm-PrA (https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/6df699f1-e23f-4245-bd10-286528c45feb/node/e042f7df-00b0-462d-b52f-e6bfe0936c0c),
 Node: N-Me-6:2 FTSAm-PrA (https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/6df699f1-e23f-4245-bd10-286528c45feb/node/6b028bf8-b5df-4467-a83c-e6da04a2e7f9),
 Node: 6:2 FTSAm-PrOH (https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/6df699f1-e23f-4245-bd10-286528c45feb/node/4d3f5a3b-8303-4ba9-960a-c47089ceafcf),
 Node: 6:2 FTSAm-Conjunct-2 (https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/6df699f1-e23f-4245-bd10-286528c45feb/node/9626f90f-9dce-42ff-8f29-bf54fe339e9d),
 Node: 6:2 FTAA (https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/6df699f1-e23f-

### How to change node names?
### how to make root node named correctly?
### why is root last in the pw.get_nodes() list?

In [62]:
# Now for related scanrios
kin_conf

,Compound,SMILES,Model,Parameter,Value,Value Unit,R2,Confidence Level_PFAS,Confidence Level_Schy,URL
0,6:2 FTNO,C[N+](C)(CCCNS(=O)(=O)CCC(C(C(C(C(C(F)(F)F)(F)...,SFO,Half-life,1.2,d,0.9976,1a,1,https://envipath.org/package/822cc087-5b9b-45d...
1,6:2 FTAA,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN(C)C)(F)F)(F)F...,SFO,Half-life,11.5,d,0.9041,2a,2,https://envipath.org/package/822cc087-5b9b-45d...
2,6:2 FTUAA,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCC/C=[N+](\C)/C)(F...,NaN,NaN,NaN,NaN,NaN,2b,2,https://envipath.org/package/822cc087-5b9b-45d...
3,6:2 FTSeAm,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCNC)(F)F)(F)F)(F...,NaN,NaN,NaN,NaN,NaN,2a,2,https://envipath.org/package/822cc087-5b9b-45d...
4,6:2 FTPrAm,FC(F)(C(C(C(C(C(CCS(=O)(=O)NCCCN)(F)F)(F)F)(F)...,NaN,NaN,NaN,NaN,NaN,2c,2,https://envipath.org/package/822cc087-5b9b-45d...
5,6:2 FTSAm-PrOH,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,NaN,NaN,NaN,NaN,NaN,2a,2,https://envipath.org/package/822cc087-5b9b-45d...
6,6:2 FTSAm-PrA,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,NaN,NaN,NaN,NaN,NaN,2a,2,https://envipath.org/package/822cc087-5b9b-45d...
7,N-Me-6:2 FTSAm-PrA,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,NaN,NaN,NaN,NaN,NaN,3a,3,https://envipath.org/package/822cc087-5b9b-45d...
8,N-Et-6:2 FTSAm,FC(F)(C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)CCS(N...,NaN,NaN,NaN,NaN,NaN,2b,2,https://envipath.org/package/822cc087-5b9b-45d...
9,6:2 FTSAm-EtOH,FC(C(F)(C(F)(C(F)(C(F)(C(F)(CCS(NCCO)(=O)=O)F)...,NaN,NaN,NaN,NaN,NaN,2b,2,https://envipath.org/package/822cc087-5b9b-45d...


In [63]:
# This set of conditions compares the names of the compounds in the table with the names + aliases of compounds in the nodes and adds related scenarios accordingly
node_list = kin_conf
for index in node_list.index:
    # print(index)
    node_info = node_list.loc[index]
    node = Node(eP.requester, id=node_info.URL)
            
    # print(node_info)
    
    ai_list_update = []

    if (not pd.isna(node_info["Confidence Level_Schy"])):
        ai1 = ConfidenceLevelAdditionalInformation()
        ai1.set_radioconfidence(node_info["Confidence Level_Schy"])
        ai_list_update.append(ai1)
        
    if (not pd.isna(node_info[["Model", "Value"]]).any()):
        ai2 = HalfLifeAdditionalInformation()
        ai2.set_lower(node_info["Value"])
        ai2.set_upper(node_info["Value"])
        ai2.set_model(node_info["Model"])
        ai_list_update.append(ai2)

    if (not len(ai_list_update) == 0):
        referral_scenario = Scenario.create(pkg, referring_scenario_id=main_scenario.get_id(), additional_information=[])
        referral_scenario.update_scenario(additional_information=ai_list_update)
        node.add_scenario(referral_scenario)
                

### deleting pathways

In [31]:
pws_to_delete = ["https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/4a86eddb-e3ed-4b1b-ae88-80b95a70af26",
                "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/92a66fe7-dadd-4b83-82f4-6f1f1a687d3a",
                "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/97ecec79-2803-4e8b-a9dc-47685394c85f"]


In [64]:
pws_to_delete =["https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/pathway/1e683118-362a-40e7-ad3e-f36b86d47803"]

In [65]:
for pw_URL in pws_to_delete:
    pw = Pathway(eP.requester, id=pw_URL)
    pw.delete()

# pw.delete()

### deleting scenarios

In [41]:
scns_to_delete = ["https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/4d575c5c-d1d3-463f-a303-c08f7ae13d6f",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/783e33ed-0a7b-4fc4-bf48-7a37fe62c4c3",
                "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/2149fd26-a24c-4c14-ba6a-f0085a2a60f6",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/1d169ee3-1aa1-4435-82cb-86ea8939aa9c",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/afbe93d8-23e8-426f-a352-fb60a1a2a604",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/ed4859e8-b65e-4c6f-93b1-863a415722bd",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/8f098770-2d77-4d27-accc-3773ac2732d2",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/2ff0a0c0-cd56-4539-8f86-410674beb20a",
                 "https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/3381330e-a56a-40be-a2af-966641ba0013"]

In [20]:
scns_to_delete = ["https://envipath.org/package/822cc087-5b9b-45da-8864-b1a27ff19d3e/scenario/cfa0edef-2e16-4b88-a596-5364c284af99"]

In [42]:
for scn_URL in scns_to_delete:
    scn = Scenario(eP.requester, id=scn_URL)
    scn.delete()